In [ ]:
import os
import re
from tqdm import tqdm
import pandas as pd
import json
from src.language_models.call_llm import ask
from src.language_models.files_loader import save_csv, load_csv

In [ ]:
def format_prompt(item):
    question = item.get("question", "")
    choices = item.get("choices") or []

    labels = ["A", "B", "C", "D", "E", "F", "G", "H"]
    used_labels = labels[:len(choices)]
    formatted_choices = [f"{lab}. {choice}" for lab, choice in zip(used_labels, choices)]

    choices_text = "\n".join(formatted_choices)  # build outside the f-string
    prompt_text = f"""Câu hỏi: {question}
Các lựa chọn:
{choices_text}

Yêu cầu:
1. Hãy phân tích đề bài và suy luận từng bước để tìm ra đáp án đúng nhất.
2. Loại trừ các phương án sai (nếu cần).
3. Cuối cùng, CHỈ IN RA dòng kết quả theo mẫu: "=== ĐÁP ÁN: [Ký tự] ==="

Bắt đầu suy luận:"""
    return prompt_text

In [ ]:
system_instruction = """Bạn là một chuyên gia AI xuất sắc với khả năng tư duy logic, toán học và phân tích đa lĩnh vực vượt trội.
Nhiệm vụ của bạn là giải quyết các câu hỏi trắc nghiệm độ khó cao.

QUY TẮC BẮT BUỘC:
1. KHÔNG đoán mò. Hãy suy luận từng bước (Step-by-step) thật cẩn thận.
2. Với bài toán/logic: Hãy viết ra các bước tính toán hoặc lập luận logic.
3. Với câu hỏi kiến thức: Hãy nhớ lại chính xác các văn bản, quy định hoặc sự kiện lịch sử.
4. Sau khi suy luận xong, hãy chốt đáp án cuối cùng theo đúng định dạng: "=== ĐÁP ÁN: [X] ===" (với X là ký tự lựa chọn)."""

In [ ]:
def process_dataset(input_file, output_file):
    print(f"--- Đang xử lý file: {input_file} ---")

    try:
        with open(input_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file {input_file}")
        return

    # Load existing CSV results (if any) into a DataFrame and collect processed ids
    processed_ids = set()
    if os.path.exists(output_file):
        try:
            prev_df = pd.read_csv(output_file, dtype=str)
            if 'qid' in prev_df.columns:
                processed_ids = set(prev_df['qid'].astype(str).tolist())
            else:
                prev_df = pd.DataFrame()
        except Exception:
            prev_df = pd.DataFrame()
    else:
        prev_df = pd.DataFrame()

    results_df = prev_df.copy()  # DataFrame of accumulated results
    buffer = []
    FLUSH_EVERY = 20  # flush to disk every N items to checkpoint

    # Vòng lặp xử lý
    for item in tqdm(data, desc="Processing"):
        qid = str(item['qid'])

        if qid in processed_ids:
            continue

        prompt = format_prompt(item)

        # Gọi API
        response = ask(model_name="vnptai_hackathon_large",
                   sys_prompt=system_instruction,
                   user_prompt=prompt,
                   num_retries=10)
        
        match = re.search(r'===\s*ĐÁP ÁN:\s*([A-Z])\s*===', response, re.IGNORECASE)
        pred = match.group(1).upper() if match else "Unknown"

        # Lưu một hàng dưới dạng dict (sau sẽ convert sang DataFrame)
        row = {
            "qid": qid,
            "answer": pred
        }
        if "answer" in item:
            row["ground_truth"] = item["answer"]

        buffer.append(row)
        processed_ids.add(qid)

        # Flush buffer to results_df and disk periodically
        if len(buffer) >= FLUSH_EVERY:
            chunk_df = pd.DataFrame(buffer)
            results_df = pd.concat([results_df, chunk_df], ignore_index=True, sort=False)
            results_df.to_csv(output_file, index=False, encoding='utf-8')
            buffer = []

    # Final flush
    if buffer:
        chunk_df = pd.DataFrame(buffer)
        results_df = pd.concat([results_df, chunk_df], ignore_index=True, sort=False)
        results_df.to_csv(output_file, index=False, encoding='utf-8')

    print(f"--- Hoàn tất! Kết quả lưu tại: {output_file} ---")
    return results_df

In [ ]:
process_dataset(input_file='data/test.json', output_file='results/test_predictions.csv')
# process_dataset(input_file='data/val.json', output_file='results/val_predictions.csv', )

--- Đang xử lý file: data/test.json ---


Processing:  87%|████████▋ | 321/370 [03:47<00:31,  1.54it/s]

In [ ]:
# df = pd.read_csv("results/val_predictions.csv")
# pred = df["answer"].astype(str).str.strip().str.upper()
# gt = df["ground_truth"].astype(str).str.strip().str.upper()
# acc = (pred == gt).mean() * 100
# print(f"Accuracy: {acc:.2f}% ({(pred == gt).sum()}/{len(df)})")

Accuracy: 33.33% (31/93)
